In [1]:
# load packages
import os
import nibabel as nib
from nilearn.image import get_data, new_img_like
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix
import pandas as pd
import numpy as np
from scipy.stats import zscore
from tqdm import tqdm

In [2]:
# load onset information
mtdat = pd.read_csv("../monologue_timestamps.csv")
mtdat.head()

,movie,start,stop,duration,type,clip,version_silenced
0,physical,0,148,148000,NIM,clip-0,0
1,physical,148,151,3000,IM,clip-1,2
2,physical,151,177,26000,NIM,clip-2,0
3,physical,177,188,11000,IM,clip-3,1
4,physical,188,245,57000,NIM,clip-4,0


In [3]:
# create events
movies = ["physical", "stutterer"]
events = {}
nm = 2
for i in range(nm):
    events[movies[i]] = mtdat.loc[mtdat.movie == movies[i],]
    events[movies[i]] = events[movies[i]].iloc[:, [1, 3, 5]]
    events[movies[i]].iloc[:, 1] = pd.Series(events[movies[i]].iloc[:, 1]/1000,dtype="int64")
    events[movies[i]].columns = ["onset","duration","trial_type"]
    events[movies[i]]

In [4]:
# perform 1st level GLMs
fdir = "M:/inarr/mri/"
ns = 28
nc = 25
for i in tqdm(range(ns)):
    ffiles = os.listdir(fdir + "sub-" + str(i+1))
    mfiles = [f for f in ffiles if "nii" in f]
    cfiles = [f for f in ffiles if "tsv" in f]
    os.makedirs("M:/inarr/mri/" + "sub-" + str(i+1) + os.sep + "contrasts", exist_ok=True)
    for j in range(nm):
        curmfile = [f for f in mfiles if movies[j] in f][0]
        curcfile = [f for f in cfiles if movies[j] in f][0]
        conf = pd.read_csv(fdir + "sub-" + str(i+1) + os.sep + curcfile, sep="\t", header=0)
        confounds = [c for c in conf.columns if not '_comp' in c]
        confounds = [c for c in confounds if not 'cosine' in c]
        confounds = [c for c in confounds if not 'motion' in c]
        conf = conf.loc[:,confounds]
        conf = conf.bfill()
        img = nib.load(fdir +  "sub-" + str(i+1) + os.sep + curmfile)
        dm = make_first_level_design_matrix(
            frame_times=np.arange(img.shape[3])*2,
            events = events[movies[j]],
            hrf_model="spm",
            high_pass=0.008,
            add_regs=conf
        )
        dm = dm.loc[:, dm.columns != 'constant']
        fmri_glm = FirstLevelModel()
        fmri_glm.fit(fdir +  "sub-" + str(i+1) + os.sep + curmfile,design_matrices=dm)
        beta_maps = []
        for cond in events[movies[j]]['trial_type'].unique():
            beta_maps.append(fmri_glm.compute_contrast(
                contrast_def=cond,
                output_type="effect_size"
            ))
        beta_data = [get_data(b) for b in beta_maps]
        beta_4d = np.stack(beta_data, axis=0) 
        beta_4d = zscore(beta_4d,0)
        beta_4d = np.nan_to_num(beta_4d, nan=0.0, posinf=0.0, neginf=0.0)
        img = new_img_like(beta_maps[0], np.moveaxis(beta_4d,0,3))
        img.to_filename("M:/inarr/mri/" + "sub-" + str(i+1) + os.sep + "contrasts" + os.sep + movies[j] + ".nii.gz")

 11%|████████▉                                                                          | 3/28 [04:18<36:26, 87.45s/it]C:\Users\mthor\AppData\Local\Temp\ipykernel_12928\1837966779.py:20: UserWarning: Matrix is singular at working precision, regularizing...
  dm = make_first_level_design_matrix(
C:\Users\mthor\AppData\Local\Temp\ipykernel_12928\1837966779.py:20: UserWarning: Matrix is singular at working precision, regularizing...
  dm = make_first_level_design_matrix(
 18%|██████████████▊                                                                    | 5/28 [07:14<33:32, 87.50s/it]C:\Users\mthor\AppData\Local\Temp\ipykernel_12928\1837966779.py:20: UserWarning: Matrix is singular at working precision, regularizing...
  dm = make_first_level_design_matrix(
C:\Users\mthor\AppData\Local\Temp\ipykernel_12928\1837966779.py:20: UserWarning: Matrix is singular at working precision, regularizing...
  dm = make_first_level_design_matrix(
 21%|█████████████████▊                               